# Advanced Urothelial Carcinoma Survival
## 1. Cohort Creation

### 1.1 Inclusion-exclusion criteria
#### a) Inclusion: patients diagnosed with advanced urothelial cancinoma between 01/01/2011 and 12/01/2021

In [3]:
setwd("~/Library/CloudStorage/Box-Box/RWD/BladderCancerSurvival/code/")
eauc <- read.csv("../Bladder/Enhanced_AdvUrothelial.csv")
eauc$AdvancedDiagnosisDate <- as.Date(eauc$AdvancedDiagnosisDate)

ptid <- subset(eauc, subset = AdvancedDiagnosisDate >= as.Date("2011-01-01") & 
                              AdvancedDiagnosisDate <= as.Date("2021-12-01"))$PatientID
length(ptid)

[1] 11276